In [13]:
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *

from mainnet_launch.constants import *
from mainnet_launch.data_fetching.get_state_by_block import *


import plotly.express as px
from multicall import Call


destinations_df = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == 1)
blocks_df = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == 1)
event_df = get_full_table_as_df(RebalanceEvents)
plan_df = get_full_table_as_df(RebalancePlans)
trasactions_df = get_full_table_as_df(Transactions)
destination_tokens_df = get_full_table_as_df(DestinationTokens)
AutopoolDestinations_df = get_full_table_as_df(AutopoolDestinations)


autopool = AUTO_USD
# maybe add a tradeable assets table?y

2025-05-30 11:32:17,747 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-30 11:32:17,747 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-30 11:32:17,748 INFO sqlalchemy.engine.Engine [cached since 1520s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x124502aa0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-30 11:32:17,748 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destinations
            WHERE destinations.chain_id = 1
        
2025-05-30 11:32:17,748

In [6]:
rebalance_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            RebalanceEvents,
        ),
        TableSelector(Transactions, join_on=(Transactions.tx_hash == RebalanceEvents.tx_hash)),
        TableSelector(
            Blocks,
            [Blocks.datetime],
            (Transactions.block == Blocks.block) & (Transactions.chain_id == Blocks.chain_id),
        ),
    ],
    where_clause=(RebalanceEvents.autopool_vault_address == AUTO_USD.autopool_eth_addr),
    order_by=Blocks.datetime,
)

tokens_df = merge_tables_as_df(
    [
        TableSelector(
            AutopoolDestinations,
        ),
        TableSelector(
            DestinationTokens,
            [DestinationTokens.token_address],
            join_on=AutopoolDestinations.destination_vault_address == DestinationTokens.destination_vault_address,
        ),
        TableSelector(
            table=Tokens,
            select_fields=[Tokens.symbol, Tokens.decimals],
            join_on=(DestinationTokens.token_address == Tokens.token_address),
        ),
    ],
    where_clause=AutopoolDestinations.autopool_vault_address == autopool.autopool_eth_addr,
)

2025-05-30 11:11:59.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-05-30 11:11:59,142 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-30 11:11:59,143 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-30 11:11:59,143 INFO sqlalchemy.engine.Engine [cached since 301.1s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x123f29060>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-30 11:11:59,143 INFO sqlalchemy.engine.Engine SELECT
    autopool_destinations.*,
    destination_tokens.token_address,
    tokens.symbol,
    tokens.decimals
FROM autopool_

2025-05-30 11:11:59.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 11:11:59.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
tokens_df

,destination_vault_address,chain_id,autopool_vault_address,token_address,symbol,decimals
0,0x04ee6c666a01018a070925040ea2aF6554d051e4,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x83F20F44975D03b1b09e64809B757c47f942BEeA,sDAI,18
1,0x53C8C211350b635269e02B2C2f1077b850C657dB,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,scrvUSD,18
2,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,crvUSD,18
3,0x9906eB64BA32Fb4fD3e5541ecA95e57610084d02,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x853d955aCEf822Db058eb8505911ED77F175b99e,FRAX,18
4,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,scrvUSD,18
...,...,...,...,...,...,...
77,0x769C6bEA7db25F4c0a52f0dB0960c42F708DBF41,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,6
78,0x9Fe20B75709403723610Fd2995D0f4A9e19C3514,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,6
79,0xEeEE628a00B0EDa99D1bc78a72Fa2F2Df4AdE2F4,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,crvUSD,18
80,0x9b18801C9E95f45AD0aDD54ef2Eb4690c44d170d,1,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6,sfrxUSD,18


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# map decimals → normalizer
dec_norm = {
    6: safe_normalize_6_with_bool_success,
    18: safe_normalize_with_bool_success,
}

autoUSD_flash_borrow_solver = "0xD02b50CFc6c2903bF13638B28D081ad11515B6f9"

balance_of_calls = [
    Call(
        row["token_address"],
        ["balanceOf(address)(uint256)", autoUSD_flash_borrow_solver],
        [row["token_address"]],
    )
    for _, row in tokens_df.iterrows()
]


before = get_state_by_one_block(balance_of_calls, block_number - 1, ETH_CHAIN)
after = get_state_by_one_block(balance_of_calls, block_number, ETH_CHAIN)

In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destination_token_values_tables import (
    _build_USD_autopool_price_calls,
    _build_ETH_autopool_price_calls,
)

In [ ]:
autoUSD_flash_borrow_solver = "0xD02b50CFc6c2903bF13638B28D081ad11515B6f9"
usdt = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
flash_borrow_usdt_bal = Call(
    usdt,
    ["balanceOf(address)(uint256)", autoUSD_flash_borrow_solver],
    [(("flash_borrow_usdt_balance"), safe_normalize_6_with_bool_success)],
)
before = get_state_by_one_block([flash_borrow_usdt_bal], 22537762 - 1, ETH_CHAIN)
after = get_state_by_one_block([flash_borrow_usdt_bal], 22537762, ETH_CHAIN)

In [17]:
tokens_df.columns

Index(['destination_vault_address', 'chain_id', 'autopool_vault_address',
       'token_address', 'symbol', 'decimals'],
      dtype='object')

In [16]:
rebalance_df.columns

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'swap_offset_period', 'tx_hash', 'block', 'chain_id', 'from_address',
       'to_address', 'effective_gas_price', 'gas_used', 'gas_cost_in_eth',
       'datetime'],
      dtype='object')